In [1]:
# Sign language to text / speech
# Dependancy Imports
import cv2
import keras
import pyttsx3
import threading 
import numpy as np
import tensorflow as tf

from gtts import gTTS 
from textblob import Word 
from nltk.corpus import words
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

In [5]:
engine = pyttsx3.init()

# Load trained model
model = keras.models.load_model("model.h5")

labels_dict = {0:'0', 1:'A', 2:'B', 3:'C', 4:'D', 5:'E', 6:'F', 7:'G', 8:'H',9:'I',10:'J',11:'K',12:'L',13:'M',14:'N',15:'O',16:'P',17:"Q",18:'R',19:'S',20:'T', 21:'U', 22:'V',23:'W',24:'X',25:'Y',26:'Z'}

In [6]:
# Helper functions
def check_word_spelling(word):
    
    word = Word(word)
    
    result = word.spellcheck()    
    return result[0][0]

def speak_letter(letter):
    engine.say(letter)
    engine.runAndWait()

In [9]:
# Detect Live feed
color_dict=(0,255,0)
# x=0
# y=0
# w=64
# h=64

img_size=128
minValue = 70

source=cv2.VideoCapture(0)


count = 0
string = " "
prev = " "
prev_val = 0

while(True):

    ret,img=source.read()

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.rectangle(img,(24,24),(250 , 250),color_dict,2)
    crop_img=gray[24:250,24:250]

    count = count + 1
    if(count % 100 == 0):
        prev_val = count

    #cv2.putText(img, str(prev_val//100), (300, 150),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,255,255),2) 
    blur = cv2.GaussianBlur(crop_img,(5,5),2)
    th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret, res = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    resized=cv2.resize(res,(img_size,img_size))
    normalized=resized/255.0
    reshaped=np.reshape(normalized,(1,img_size,img_size,1))
    result = model.predict(reshaped)

    label=np.argmax(result,axis=1)[0]

    if(count == 300):
        count = 99
        prev= labels_dict[label]
        if(label == 0):
            string = string + " "
            #check spell
            string = check_word_spelling(string)
            speak_letter(string)
        else:
            string = string + prev
    
    # cv2.putText(img, prev, (24, 14),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2) 
    cv2.putText(img, string, (275, 50),cv2.FONT_HERSHEY_SIMPLEX,0.8,(200,200,200),2)
    cv2.imshow("Binary Screen",res)    
    cv2.imshow('LIVE Detection',img)
    key=cv2.waitKey(1)
    
 
    if(key==27):#press Esc. to exit
        break

print(f'Detected String: ',string)        
cv2.destroyAllWindows()
source.release()
cv2.destroyAllWindows()

get_ipython().system('pip install gTTS')
        

Detected String:   IZZTWH 


You should consider upgrading via the 'C:\Users\kabra\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
